# Transformer
前面学习了RNN和LSTM以及Seq2Seq，同时也接触了不同类型的注意力，现在要学习一个新的架构，它是后面要学习的Bert和GPT的核心-Transformer。  
Transformer的核心就是自注意力机制，它能够为输入序列中的每个元素分配不同的权重，从而更好地捕捉序列内部的依赖关系。同时其抛弃了RNN和LSTM中的循环结构，采用了全新的编码器-解码器架构，使得模型可以并行处理输入数据，进一步加速训练过程，提高计算效率。  
Transformer架构图:  
<img src="./images/Transformer架构图.png" alt="examples" style="zoom:45%;" />

## Transformer组件
根据上面的架构图，逐个组建去实现。  
1.多头自注意力：通过ScaledDotProductAttention类实现缩放点积注意力机制，然后通过MultiHeadAttention类实现多头自注意力机制  
2.逐位置前馈网络：通过PoswiseFeedForwardNet类实现逐位置前馈网络  
3.正弦位置编码表：通过get_sin_code_table函数生成正弦位置编码表  
4.填充掩码：通过get_attn_pad_mask函数为填充令牌`<pad>`生成注意力掩码，避免注意力机制关注无用的信息  
5.编码器层：通过EncoderLayer类定义编码器的单层  
6.编码器：通过Encoder类定义Transformer完整的编码器部分  
7.后续掩码：通过get_attn_subsequent_mask函数为后续令牌(当前位置后面的信息)生成注意力掩码，避免解码器中的注意力机制"偷窥"未来的目标数据  
8.解码器层：通过DecoderLayer类定义解码器的单层  
9.解码器：通过Decoder类定义Transformer完整的解码器部分  
10.Transformer类：此类将编码器和解码器整合为完整的Transformer模型

### 组件1 多头自注意力(包含残差连接和层归一化)
首先实现Transformer的核心组件，多头自注意力。主要有两个子组件，缩放点积注意力类和多头自注意力类

#### 缩放点积注意力(加入掩码机制)

<img src="./images/scaled_dot_attention_mask.png" alt="examples" style="zoom:45%;" />

掩码机制的说明:  
这里加入的掩码机制主要是填充注意力掩码：当处理变长序列时，通常会对较短序列进行填充，使所有序列的长度一致，一边进行批量处理。  
但是填充值是没有意义的，所以需要通过掩码机制将其设置为极小值，这样在应用softmax时填充位置的权重将接近于0，不会对模型产生影响。 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
d_k = 64 
d_v = 64

# 定义缩放点积注意力函数
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()
    def forward(self, Q, K, V, attn_mask):
        #------------------------- 维度信息 --------------------------------
        # Q K V [batch_size, n_heads, seq_len, seq_dim] -> fea_dim = n_heads * seq_dim
        # attn_mask [batch_size, n_heads, len_q, len_k]
        
        # 计算注意力分数（原始权重）[batch_size, n_heads, len_q, len_k]
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        
        # 使用注意力掩码，将attn_mask中值为1的位置(填充位置)的权重替换为极小值
        scores.masked_fill_(attn_mask, -1e9)
        
        # 对注意力分数进行softmax归一化处理
        # weight -> [batch_size, n_heads, len_q, len_k]
        weight = nn.Softmax(dim=-1)(scores)
        
        # 再次点积操作，计算上下文向量(注意力的输出)
        # context -> [batch_size, n_heads, len_q, dim_v]
        context = torch.matmul(weight, V)
        return context, weight      

#### 多头自注意力类(残差连接和层归一化)

<img src="./images/multi_head_attention.png" alt="examples" style="zoom:45%;" />

残差连接:  
残差连接是一种在神经网络中广泛使用的技术，用于加快网络的训练和提高模型的性能。在残差网络中，每个层都添加了一个跨层连接，可以将输入数据直接连接到输出数据，也可以将输入数据直接传传递到后续层次，从而提高信息的传递效率和网络的训练速度。  
其实就是将输入与输出直接相加然后再通过激活函数。

层归一化:  
层归一化是一种正则化技巧，用于缓解神经网络中的内部协变量偏移问题。在Transformer模型中，层归一化通常应用于残差连接之后，用于对输出进行归一化。

ref:  
https://zhuanlan.zhihu.com/p/620297938  
https://zhuanlan.zhihu.com/p/353062927